In [8]:
import boto3
import json
import os
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess


#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = 'us-east-1'

print(region)
sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::829250931565:role/service-role/AmazonSageMakerAdminIAMExecutionRole_1"


# retrieve sklearn image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="1.4-2",
    py_version="py3",
    instance_type="ml.m5.large"
)

#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

#Upload tar.gz to bucket
model_path="s3://amazon-sagemaker-829250931565-us-east-1-59r7k7fax6wn1j/shared/model.tar.gz"

#Step 1: Model Creation
model_name = "qyang-healthcare-" + strftime("%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_path,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_path,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])


#Step 2: EPC Creation
sklearn_epc_name = "qyang-healthcare-" + strftime("%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])


#Step 3: EP Creation
endpoint_name = "qyang-healthcare-ep" + strftime("%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)


us-east-1
sagemaker-us-east-1-829250931565
Model name: qyang-healthcare-23-19-03
Model Arn: arn:aws:sagemaker:us-east-1:829250931565:model/qyang-healthcare-23-19-03
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:829250931565:endpoint-config/qyang-healthcare-23-19-04
Endpoint Arn: arn:aws:sagemaker:us-east-1:829250931565:endpoint/qyang-healthcare-ep23-19-04
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'qyang-healthcare-ep23-19-04', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:829250931565:endpoint/qyang-healthcare-ep23-19-04', 'EndpointConfigName': 'qyang-healthcare-23-19-04', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.4-2-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:1f681fc3b9